In [1]:
from class_definitions import *

In [2]:
kcw_wines = pd.read_csv('kings_county_wines.csv',index_col=0)

In [3]:
wine_names = list(kcw_wines['name'])

In [4]:
wine = Wine(wine_names[0])

In [5]:
wine.name

'Mâcon-Loche 2018 Clos des Rocs'

In [6]:
wine.search_terms

['macon', 'loche', 'clos', 'des', 'rocs']

In [7]:
wine.__dict__

{'source': 'name-search',
 'name': 'Mâcon-Loche 2018 Clos des Rocs',
 'search_terms': ['macon', 'loche', 'clos', 'des', 'rocs'],
 'vintage': '2018',
 'wineSearcher': {'url': 'https://www.wine-searcher.com/find/macon+loche+clos+des+rocs+2018',
  'Producer': 'Domaine Clos des Rocs',
  'Region/Appellation': 'Macon Loche',
  'Grape/Blend': 'Chardonnay',
  'Food Suggestion': 'Chicken and Turkey',
  'Wine Style': 'White - Buttery and Complex',
  'Alcohol Content': '12.5 - 13.5%',
  'Notes': 'null',
  'name': '2018 Domaine Clos des Rocs Macon-Loche En Pres Foret, Burgundy, France'}}

In [8]:
bs = wine._scrape_vivino()

In [9]:
wine._get_searchURL('https://www.vivino.com/search/wines?q=')

'https://www.vivino.com/search/wines?q=macon+loche+clos+des+rocs+2018'

In [10]:
#possible matches 
wine_cards = bs.find_all(attrs={'class':'wine-card__content'})

In [11]:
wine_card = wine_cards[0]

In [12]:
# links to wines 
[card.find_all('a')[0]['href'] for card in wine_cards]

['/wines/4844313', '/wines/162577570', '/wines/4844309']

In [13]:
possible_matches = []
for card in wine_cards:
    d = {}
    d['wine_link'] = card.find('a')['href']
    d['wine_name'] = card.find('a').getText().replace('\n','')
    
    averages = card.find('div',attrs={'class':"text-color-alt-gray wine-card__averages"})
    average_rating = averages.find(
        attrs={'class':"text-inline-block light average__number"})\
    .getText(strip=True)
    d['average_rating']=float(average_rating)
    
    p_tags = averages.find_all('p')
    text = [p.getText(strip=True) for p in p_tags]
    for t in text:
        if 'rating' in t:
            pattern = re.compile("[0-9]")
            temp = re.findall(r'\d+',t)
            ratings = list(map(int,temp))[0]
            d['ratings_count'] = ratings
    possible_matches.append(d)
print(possible_matches)

[{'wine_link': '/wines/4844313', 'wine_name': 'Clos des Rocs Mâcon-Loché', 'average_rating': 3.7, 'ratings_count': 74}, {'wine_link': '/wines/162577570', 'wine_name': 'Clos des Rocs En Charpaux Mâcon-Loché', 'average_rating': 0.0, 'ratings_count': 1}, {'wine_link': '/wines/4844309', 'wine_name': 'Clos des Rocs En Près Forêt Mâcon-Loché', 'average_rating': 3.8, 'ratings_count': 440}]


In [14]:
wine_card.find('a').getText().replace('\n','')

'Clos des Rocs Mâcon-Loché'

In [15]:
wine_page = requests.get("https://www.kingscountywines.com/albarino-rias-baixas-2017-columna.html")
wine_bs = BeautifulSoup(wine_page.content,features='lxml')
wine_dict = {}

wine_dict['name']=wine_bs.find('h1').getText()

offer = wine_bs.find('div',{'itemprop':'offers'})
price = float(offer.find('span',{'itemprop':'price'}).getText())
wine_dict['price']=price

wine_dict['description'] = offer.p.getText()

image_link = wine_bs.find('img',attrs={'itemprop':'image'})
if image_link != None:
    wine_dict['image_link'] = image_link['src']
else:
    wine_dict['image_link'] = 'null'

in_stock = wine_bs.find('table',attrs={'class':'details'})
if in_stock != None:
    in_stock = in_stock.find_all('td')[-1].getText().split(' \n')[0]
else:
    in_stock = 'null'
wine_dict['in_stock']=in_stock

details_text = [x.getText() for x in wine_bs.find('div',attrs={'class':'page info active'}).find_all('p')]
details = "".join([x for x in details_text if x != ' ' and x != '\xa0'])
wine_dict['details'] = details

In [16]:
wine_dict

{'name': 'Albariño Rias Baixas 2019 Columna',
 'price': 18.99,
 'description': 'Saline & mineral, with peach & pear high notes. Essential.',
 'image_link': 'https://cdn.shoplightspeed.com/shops/603808/files/19164536/albarino-rias-baixas-2019-columna.jpg',
 'in_stock': 'In stock',
 'details': ''}

In [17]:
wine_bs.find('h1').getText()

'Albariño Rias Baixas 2019 Columna'

In [21]:
KCW_wine = Wine(KCW_link="https://www.kingscountywines.com/albarino-rias-baixas-2017-columna.html")

In [22]:
KCW_wine.__dict__

{'source': 'KCW',
 'KCW': {'KCW_name': 'Albariño Rias Baixas 2019 Columna',
  'price': 18.99,
  'description': 'Saline & mineral, with peach & pear high notes. Essential.',
  'image_link': 'https://cdn.shoplightspeed.com/shops/603808/files/19164536/albarino-rias-baixas-2019-columna.jpg',
  'in_stock': 'In stock',
  'details': ''},
 'name': 'Albariño Rias Baixas 2019 Columna',
 'search_terms': ['albarino', 'rias', 'baixas', 'columna'],
 'vintage': '2019',
 'wineSearcher': {'url': 'https://www.wine-searcher.com/find/albarino+rias+baixas+columna+2019',
  'Producer': 'Compania de Vinos del Atlantico',
  'Region/Appellation': 'Rias Baixas',
  'Grape/Blend': 'Albarino (Alvarinho)',
  'Food Suggestion': 'Shellfish, Crab and Lobster',
  'Wine Style': 'White - Green and Flinty',
  'Alcohol Content': '12.5%',
  'Notes': 'Ownership: Compania de Vinos del Atlantico.',
  'name': '2019 Columna Albarino, Rias Baixas, Spain'}}

In [19]:
import joblib

In [24]:
joblib.dump(KCW_wine,f'Wines/{KCW_wine.name}')

['Wines/Albariño Rias Baixas 2019 Columna']